Check the status of tube lines

http://docs.python-requests.org/en/master/

http://docs.python-requests.org/en/master/user/quickstart/#make-a-request Tutorial

https://github.com/timcnicholls/transport_api_demo/blob/master/harwell_wantage_bus.py

23-6 Removed Goodness attribute as just duplicates state

In [1]:
import requests
import requests_mock
import json
import pprint
import unittest
import sys

In [2]:
def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

In [3]:
lines = ['Bakerloo',
 'Central',
 'Circle',
 'District',
 'Hammersmith & City',
 'Jubilee',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'Victoria',
 'Waterloo & City']

In [4]:
ls

Tube requests 26-6-2017.ipynb  hammersmith-city.json
bakerloo.json                  tube_state.json
circle.json                    victoria.json


In [5]:
len(lines)

11

# Basic function of API

In [6]:
url = 'https://api.tfl.gov.uk/line/mode/tube/status'
#data = requests.get(url).json() # get the response object

# Cached data
with requests_mock.mock() as m:
    m.get(url, text=open('tube_state.json').read())
    data = requests.get(url).json()
    
print_json(len(data))
print_json(data[0]['name'])
print_json(data[0]['lineStatuses'][0]['statusSeverityDescription'])

11
'Bakerloo'
'Good Service'


In [7]:
data_dict = dict.fromkeys(lines)    # Create the empty dict for the data
data_dict

{'Bakerloo': None,
 'Central': None,
 'Circle': None,
 'District': None,
 'Hammersmith & City': None,
 'Jubilee': None,
 'Metropolitan': None,
 'Northern': None,
 'Piccadilly': None,
 'Victoria': None,
 'Waterloo & City': None}

Main logic for parsing the data into a useful dict

In [8]:
for line in data:                     # Assign data via key
    statuses = [status['statusSeverityDescription'] for status in line['lineStatuses']]
    state = ' + '.join(sorted(set(statuses)))
    
    if state == 'Good Service':   # if good status, this is the only status returned
        reason =  'Nothing to report'
    else:
        reason = ' *** '.join([status['reason'] for status in line['lineStatuses']])

    attr = {'State': state, 'Description': reason}
    data_dict[line['name']] = attr
    
print_json(data_dict)

{'Bakerloo': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Central': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Circle': {'Description': 'Circle Line: No service Edgware Road to Aldgate '
                           'via Embankment while we fix a signal systems '
                           'failure.  London Buses, London Overground, Great '
                           'Northern accepting valid  London Underground '
                           'tickets via any reasonable route. ',
            'State': 'Part Suspended'},
 'District': {'Description': 'District Line: No service Whitechapel to '
                             'Richmond, Ealing Broadway and Wimbledon and '
                             'Between Wimbledon and Edgware Road. SEVERE '
                             'DELAYS Whitechapel to Upminster while we fix a '
                             'signal systems failure at Earls Court. London '
                             'Buses, London Overground, 

# Create tube class

Wrap in a class with a method for updating and another for returning line of interest. Follow https://github.com/timcnicholls/home-assistant/blob/transport-api/homeassistant/components/sensor/uk_transport.py

In [9]:
class LondonTubeSensor():    # Entity
    """
    Sensor that reads the status of a tube lines using the TFL API.
    """
    API_URL_BASE = "https://api.tfl.gov.uk/line/{}/status"
    ICON = 'mdi:subway'

    def __init__(self, name):
        """Initialize the sensor."""
        self._name = name             # the name of the line from the allowed list
        self._data = {}
        self._url = self.API_URL_BASE
        self._state = 'Updating'
        self._description = ['Updating']

    @property
    def name(self):
        """Return the line name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    @property
    def device_state_attributes(self):
        """Return other details about the sensor state."""
        attrs = {}  # {'attribution': 'Data provided by transportapi.com'}
        attrs['Description'] = self._description # if there is data, append
        return attrs

    def update(self):
        """Perform an API request and update the sensor."""
        response = requests.get(self._url.format(self._name.lower()))    # make a request

        if response.status_code != 200:
            #_LOGGER.warning("Invalid response from API")
            print("Invalid response from API")

        else:
            self._data = response.json()[0]['lineStatuses']   # convert to json and get statuses list
            statuses = [status['statusSeverityDescription'] for status in self._data]   # get all statuses on a line

            if 'Good Service' in statuses:   # if good status, this is the only status returned
                self._state = 'Good Service'
                self._description = 'Nothing to report'
            else:
                self._state = ' + '.join(sorted(set(statuses)))   # get the unique statuses and join
                self._description = [status['reason'] for status in self._data] # get the reasons

## Unit test the class
Edge cases: good service and bad.
Sometimes state is reverse order

In [10]:
## Mock data
with requests_mock.mock() as m:
    url = 'https://api.tfl.gov.uk/line/bakerloo/status'
    m.get(url, text=open('bakerloo.json').read())
    sensor = LondonTubeSensor('Bakerloo')
    sensor.update()

class TestLondonTubeSensor(unittest.TestCase):

    def test_of_good_state(self):
        self.assertEqual(sensor.state,'Good Service')
        
    def test_of_good_description(self):    
        self.assertEqual(sensor._description, 'Nothing to report')

        
unittest.main(argv=['ignored', '-v'], exit=False)

test_of_good_description (__main__.TestLondonTubeSensor) ... ok
test_of_good_state (__main__.TestLondonTubeSensor) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [11]:
## Mock data
CONF_LINE= 'line' 
config = {CONF_LINE:['Hammersmith-city']}

with requests_mock.mock() as m:
    url = 'https://api.tfl.gov.uk/line/hammersmith-city/status'
    m.get(url, text=open('tube_state.json').read())   # tube_state.json contains the Hammersmith-city data
    sensor = LondonTubeSensor(config[CONF_LINE][0])
    sensor.update()

class TestLondonTubeSensor(unittest.TestCase):

    def test_of_bad_state(self):
        self.assertEqual(sensor.state,'Minor Delays + Part Closure')  # Sometimes comes out in other order
        
    def test_of_bad_description(self):    
        self.assertEqual(sensor._description, ['Ongoing investigations by Rob', 'More investigations'])
    
    def test_name(self):
        self.assertEqual(sensor.name, 'Hammersmith-city')

        
unittest.main(argv=['ignored', '-v'], exit=False)

test_name (__main__.TestLondonTubeSensor) ... ok
test_of_bad_description (__main__.TestLondonTubeSensor) ... FAIL
test_of_bad_state (__main__.TestLondonTubeSensor) ... FAIL

FAIL: test_of_bad_description (__main__.TestLondonTubeSensor)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-11-96e545941aeb>", line 17, in test_of_bad_description
    self.assertEqual(sensor._description, ['Ongoing investigations by Rob', 'More investigations'])
AssertionError: 'Nothing to report' != ['Ongoing investigations by Rob', 'More investigations']

FAIL: test_of_bad_state (__main__.TestLondonTubeSensor)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-11-96e545941aeb>", line 14, in test_of_bad_state
    self.assertEqual(sensor.state,'Minor Delays + Part Closure')  # Sometimes comes out in other order
AssertionError: 'Good Service' != 'Minor Delay

## Unused

In [12]:
url = 'https://api.tfl.gov.uk/line/{}/status'.format('victoria')
response = requests.get(url)  # get the response object
filename = 'victoria.json'

with open(filename, 'wb') as fd:                 # write the response to file. Can format the .json file but no blank lines allowed 
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)

data = response.json()[0]['lineStatuses']       # data is a list of statuses on the line
#print_json(data)